In [1]:
from langchain.schema import Document
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import re

class HTMLToTextWithIndentation:
    def __init__(self):
        self.indentation_map = {
            'h1': 0,
            'h2': 1,
            'h3': 2,
            'h4': 3,
            'h5': 4,
            'h6': 5,
            'p': 0,
            'ul': 1,
            'ol': 1,
            'li': 2  # 목록 항목의 들여쓰기 수준을 늘려서 조정합니다.
        }

    def transform_document(self, document: Document) -> Document:
        soup = BeautifulSoup(document.page_content, 'html.parser')
        text = self.process_element(soup.body if soup.body else soup)
        text = re.sub(r'\n+', '\n', text).strip()
        return Document(page_content=text, metadata=document.metadata)

    def process_element(self, element, current_indent=''):
        if element.name is None:
            text = element.strip()
            if text:
                return current_indent + text + '\n'
            return ''
        
        if element.name == 'footer':
            return ''

        if element.name == 'table':
            return current_indent + self.html_table_to_markdown(element) + '\n'

        content = []
        new_indent = self.calculate_indent(element, current_indent)

        if element.name == 'li':
            list_prefix = '• ' if 'ul' in element.parent.name else '1. '
            content.append(new_indent + list_prefix + self.get_combined_text(element).strip() + '\n')
        elif element.name == 'p':
            content.append(new_indent + self.get_combined_text(element).strip() + '\n')
        else:
            for child in element.children:
                child_content = self.process_element(child, new_indent)
                if child_content.strip():
                    content.append(child_content)

        return ''.join(content)

    def get_combined_text(self, element):
        """자식 요소들이 분리된 텍스트를 하나로 병합"""
        if element.name is None:
            return element.strip()
        combined_text = ''
        for child in element.children:
            combined_text += self.get_combined_text(child)
        return combined_text

    def calculate_indent(self, element, current_indent):
        if element.name in self.indentation_map:
            indent_level = self.indentation_map[element.name]
            return ' ' * (indent_level * 4)  # 들여쓰기 수준을 4칸 단위로 적용합니다.
        else:
            return current_indent

    def html_table_to_markdown(self, table):
        html_string = str(table)
        html_io = StringIO(html_string)
        df = pd.read_html(html_io)[0]
        return df.to_markdown(index=False)

    def transform_documents(self, documents):
        combined_text = ""
        for document in documents:
            transformed_doc = self.transform_document(document)
            combined_text += transformed_doc.page_content + "\n\n"
        return combined_text.strip()

# HTML 파일에서 텍스트 변환 예시:
files = [
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_1.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_2.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_3.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_4.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_5.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_6.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_7.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_8.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_9.html',
    './processed_txt/IRP자산관리_약관(기업형)_20240401_P24_Page_10.html',
]

documents = []
for file_path in files:
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    document = Document(page_content=html_content, metadata={"source": file_path})
    documents.append(document)

transformer = HTMLToTextWithIndentation()
combined_text = transformer.transform_documents(documents)
print(combined_text)


⑨
에는 지급기일까지 계산된 적립금에 지급기일의 다음날부터 지급일까지의 기간에 대하여 시행령 제1 1조에서 정한 이율을 적용하여 연단위 복리로 계산한 금액을 추가로 지급합니다. 단, 실적배당형 상품으로 운용되었을 경우 지급기일시점에 정상 지급되었어야 할 금액(이하 "정상 처리시 지급액"이라 합니다)이"실제 지급액"보다 많은 때에는 그 차액을 더하여 보상합니다.제8항에도 불구하고 관련 법령에서 예외로 인정하는 사유에 해당하는 경우, 제27조(펀드의 선택 및 변경) 제5항에 해당하는 경우 등 회사의 귀책사유가 없는경우에는 제8항을 적용하지 않습니다.
제15조 (양도 · 압류 · 담보제공 및 중도인출)
1. ① 이 제도의 급여를 받을 권리는 양도 또는 압류하거나 담보로 제공할 수 없습니
2. 다.
3. ② 제1 항에도 불구하고, 가입자는 법 제7조에서 정하거나 법 제24조에서 정한 경
4. 우로서 운용관리기관의 통지가 있는 경우에는 법령에서 정하는 한도 이내에서
5. 담보제공 또는 중도인출할 수 있습니다.
제16조 (중도해지)
1. ① 가입자는 근로자대표의 동의를 얻어 이 계약을 해지할 수 있습니다.
2. ② 회사는 다음 각 호의 하나에 해당하는 경우 이 계약을 해지할 수 있습니다.
1. 1. 사용자가 파산 또는 폐업된 경우
2. 2. 사용자 또는 가입자의 계약 관련 서류 기재내용상 중요부분에 허위사실이 있
3. 는 경우
4. 3. 관련 법령 등에 의하여 해지가 불가피한 경우
5. ③ 제2항 각호의 사유가 발생하여 회사가 이 계약을 해지하고자 할 경우에는 사용
6. 자 및 가입자에게 1개월 이전에 서면통지를 하여야 합니다.
7. ④ 다음 각 호의 하나의 사유로 인하여 중도해지 할 경우에는 특별중도해지로 처
8. 리합니다.
9. 1. 사용자가 영위하는 사업장의 합병, 영업양도로 인하여 가입자가 근로자대표의
10. 동의를 얻어 해지를 요청한 경우
11. 2. 제2항 제1호에 해당하는 경우
12. 3. 관련 법령 등에 의하여 해지가 불가피한 경우
13. 4. 가입자가 퇴